In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))




[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aadesh/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
data = pd.read_csv('dataset/amazon_alexa.tsv', delimiter='\t', quoting= 3)
print(f"Dataset Shape: {data.shape}")

Dataset Shape: (3150, 5)
